In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install ultralytics

Looking in indexes: https://download.pytorch.org/whl/cpu


In [3]:
import cv2
from ultralytics import YOLO
import signal

# Load the YOLO model (ensure your model supports person detection)
model = YOLO(r'C:\Users\Hp\yolov5/yolo11n-pose.pt')

# Open the webcam (use 0 for the default webcam, or replace with the correct camera index)
cap = cv2.VideoCapture(0)

# Flag to indicate running state
running = True

# Define the signal handler for a graceful kill switch
def signal_handler(sig, frame):
    global running
    print("Kill switch activated! Exiting gracefully...")
    running = False

# Register the signal handler for SIGINT (Ctrl+C)
signal.signal(signal.SIGINT, signal_handler)

try:
    while running:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break

        # Perform object detection on the frame
        results = model(frame)  # Run YOLO on the current frame
        annotated_frame = frame.copy()  # Copy frame for annotations

        for detection in results[0].boxes.data:
            # Each detection is in the format [x1, y1, x2, y2, confidence, class_id]
            x1, y1, x2, y2, confidence, class_id = detection.cpu().numpy()
            if int(class_id) == 0:  # Assuming class_id 0 corresponds to "person"
                # Draw a rectangle around the detected person
                cv2.rectangle(
                    annotated_frame, 
                    (int(x1), int(y1)), 
                    (int(x2), int(y2)), 
                    (0, 255, 0), 
                    2
                )
                # Add label and confidence score
                label = f"Person: {confidence:.2f}"
                cv2.putText(
                    annotated_frame, 
                    label, 
                    (int(x1), int(y1) - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, 
                    (0, 255, 0), 
                    2
                )

        # Display the annotated frame
        cv2.imshow("Person Detection", annotated_frame)

        # Check for ESC key to exit
        if cv2.waitKey(1) & 0xFF == 27:  # 27 is the ASCII code for the ESC key
            print("ESC key pressed! Exiting...")
            break

finally:
    # Ensure the camera is released and all windows are closed
    cap.release()
    cv2.destroyAllWindows()
    print("Camera released, windows closed.")



0: 480x640 1 person, 3188.5ms
Speed: 301.2ms preprocess, 3188.5ms inference, 370.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 152.2ms
Speed: 47.5ms preprocess, 152.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 193.3ms
Speed: 43.1ms preprocess, 193.3ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 132.6ms
Speed: 3.0ms preprocess, 132.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 142.4ms
Speed: 4.0ms preprocess, 142.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 118.4ms
Speed: 3.0ms preprocess, 118.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 130.9ms
Speed: 3.1ms preprocess, 130.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 165.6ms
Speed: 4.0ms preprocess, 165.6ms inference, 3.8ms postprocess per 

In [3]:
import cv2
from ultralytics import YOLO
import time

# Load the YOLO model
model = YOLO(r'C:\Users\Hp\yolov5/yolo11n-pose.pt')

# Open the webcam (use 0 for the default webcam, or replace with the correct camera index)
cap = cv2.VideoCapture(0)

# Check if webcam is opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

time.sleep(2)  # Wait 2 seconds

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Run the YOLO model on the frame
    results = model(frame, verbose=False)

    # Draw bounding boxes for detected persons
    for r in results:
        bound_box = r.boxes.xyxy  # Bounding boxes
        conf = r.boxes.conf.tolist()  # Confidence scores

        for index, box in enumerate(bound_box):
            if conf[index] > 0.75:  # Confidence threshold
                x1, y1, x2, y2 = map(int, box.tolist())
                # Draw the bounding box and label on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, "Person", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the processed frame
    cv2.imshow("Webcam Feed", frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [2]:
import cv2
from ultralytics import YOLO
import pandas as pd

# Correct path to the model file
model = YOLO(r'C:\Users\Hp\yolov5/yolo11n-pose.pt')

# Video path (ensure this is correct)
video_path = r"C:\Users\Hp\Downloads\vid1st.mp4"
cap = cv2.VideoCapture(video_path)

# Get video frame count and FPS
frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
seconds = round(frames / fps)

# Set number of frames you want to process
frame_total = 500
i = 0
a = 0

# List to store all data
all_data = []

while cap.isOpened():
    # Set position to capture the frame based on the total number of frames
    cap.set(cv2.CAP_PROP_POS_MSEC, (i * ((seconds / frame_total) * 1000)))
    flag, frame = cap.read()

    if not flag:
        break

    # Save the image frame
    image_path = f"C:/Users/Hp/Desktop/images/img_{i}.jpg"
    cv2.imwrite(image_path, frame)

    # YOLO detection on the frame
    results = model(frame, verbose=False)

    for r in results:
        bound_box = r.boxes.xyxy  # Bounding boxes
        conf = r.boxes.conf.tolist()  # Confidence
        keypoints = r.keypoints.xyn.tolist()  # Keypoints

        for index, box in enumerate(bound_box):
            if conf[index] > 0.75:  # Threshold to filter detections
                x1, y1, x2, y2 = box.tolist()
                pict = frame[int(y1):int(y2), int(x1):int(x2)]
                output_path = f"C:/Users/Hp/Desktop/newfolder/person_{a}.jpg"

                # Prepare data for CSV
                data = {'image_name': f'person_{a}.jpg'}
                for j in range(len(keypoints[index])):
                    data[f'x{j}'] = keypoints[index][j][0]
                    data[f'y{j}'] = keypoints[index][j][1]

                # Append the data to the list
                all_data.append(data)

                # Save the detected person image
                cv2.imwrite(output_path, pict)
                a += 1

    i += 1

# Release video capture
cap.release()
cv2.destroyAllWindows()

# Create a DataFrame and save to CSV
df = pd.DataFrame(all_data)
csv_file_path = "C:/Users/Hp/Desktop/aaa/keypoints.csv"
df.to_csv(csv_file_path, index=False)

# Print summary
print(f"Processed {i-1} frames and saved {a-1} persons.")


KeyboardInterrupt: 

In [ ]:
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()


In [ ]:
!pip show torch

In [ ]:
pip install --upgrade torch


In [ ]:
import torch
print(torch.__version__)


In [ ]:
ls | grep torch


In [ ]:
pip uninstall torch torchvision torchaudio -y


In [ ]:
!dir . | findstr "torch"
